In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from ydata_profiling import ProfileReport
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from sklearn.preprocessing import StandardScaler
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline

## EDA

In [ ]:
df1 = pd.read_csv('D:\\NCKH - Estimation GHG\\Output\\2020\\data_co2_modis_soilgrid_2020.csv')
df2 = pd.read_csv('D:\\NCKH - Estimation GHG\\Output\\2021\\data_co2_modis_soilgrid_2021.csv')
df3 = pd.read_csv('D:\\NCKH - Estimation GHG\\Output\\2022\\data_co2_modis_soilgrid_2022.csv')
df = pd.concat([df1, df2, df3], ignore_index=True)
df.info()

### Create Time Series Feature

In [ ]:
df = df.drop(columns=['LST_Night_Terra_C', 'LST_Night_Aqua_C', 'LST_Day_Terra_C', 'LST_Day_Aqua_C'])
df['time'] = pd.to_datetime(df['time'])
df['date'] = pd.to_datetime(df['date'])
df.head()
df.info()

In [ ]:
# Feature chu kỳ
df['month'] = df['time'].dt.month
df['day_of_year'] = df['time'].dt.dayofyear
df['day_of_week'] = df['time'].dt.dayofweek
df['hour'] = df['time'].dt.hour

# Mã hóa chu kỳ bằng sin/cos
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
df['dayofyear_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365.25)
df['dayofyear_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365.25)

# Feature xu hướng
df['year'] = df['time'].dt.year
df['time_idx'] = (df['time'] - df['time'].min()).dt.total_seconds() / (3600 * 24)

In [ ]:
time_diff = df['time'].diff()  
pass_threshold = pd.Timedelta(minutes=30)   
df['pass_id'] = (time_diff > pass_threshold).cumsum()
print("Dữ liệu sau khi gán 'pass_id':")
print(df[['time', 'pass_id']])
print("\n")

In [ ]:
# Khoảng cách Haversine (khoảng cách giữa 2 điểm trên mặt cầu)
def haversine_distance(lon1, lat1, lon2, lat2):
    R = 6371  
    lon1_rad, lat1_rad, lon2_rad, lat2_rad = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

In [ ]:
df['xco2_lag_1'] = df.groupby('pass_id')['xco2'].shift(1)
df['xco2_lag_2'] = df.groupby('pass_id')['xco2'].shift(2)
df['xco2_lag_3'] = df.groupby('pass_id')['xco2'].shift(3)

In [ ]:
window_size = 5
df['xco2_rolling_mean_5'] = df.groupby('pass_id')['xco2'].transform(
    lambda x: x.rolling(window=window_size, min_periods=1).mean()
)
df['xco2_rolling_std_5'] = df.groupby('pass_id')['xco2'].transform(
    lambda x: x.rolling(window=window_size, min_periods=1).std()
)

window_size = 10
df['xco2_rolling_mean_10'] = df.groupby('pass_id')['xco2'].transform(
    lambda x: x.rolling(window=window_size, min_periods=1).mean()
)
df['xco2_rolling_std_10'] = df.groupby('pass_id')['xco2'].transform(
    lambda x: x.rolling(window=window_size, min_periods=1).std()
)

### BASIC: Create time window, skip space

In [ ]:
df_indexed = df.set_index('time')
windows = ['7D', '30D']
for window in windows:
    rolling_xco2 = df_indexed['xco2'].rolling(window, closed='left')
    df[f'xco2_regional_mean_{window}'] = rolling_xco2.mean().values
    df[f'xco2_regional_std_{window}'] = rolling_xco2.std().values
    
print(df.to_string())

### ADVANCE: Create a time-space weighted average window

In [ ]:
def calculate_spatiotemporal_weighted_features(target_row, all_data, config):
    """
    Tính toán các đặc trưng trung bình có trọng số theo không gian-thời gian.
    
    Args:
        target_row: Hàng dữ liệu (điểm P) mà ta muốn tính đặc trưng.
        all_data: Toàn bộ DataFrame chứa các điểm lân cận tiềm năng.
        config: Dictionary chứa các tham số.
    
    Returns:
        Một dictionary chứa các đặc trưng mới.
    """
    p_time = target_row['time']
    p_lat = target_row['latitude']
    p_lon = target_row['longitude']
    
    # 1. Lọc các điểm lân cận trong một cửa sổ thời gian
    past_data = all_data[
        (all_data['time'] >= p_time - config['time_window']) & 
        (all_data['time'] < p_time)
    ]
    if past_data.empty:
        return {'xco2_weighted_mean_idw': np.nan, 'xco2_weighted_mean_gaussian': np.nan}
        
    # 2. Tính toán khoảng cách không gian và thời gian
    space_dist = haversine_distance(p_lon, p_lat, past_data['longitude'], past_data['latitude'])
    time_dist_days = (p_time - past_data['time']).dt.total_seconds() / (3600 * 24)
    values = past_data['xco2']
    
    # 3. Tính toán trọng số    
    # a. Nghịch đảo Khoảng cách (IDW)
    weight_idw = 1.0 / (space_dist * time_dist_days + config['epsilon'])
    
    # b. Gaussian Kernel
    weight_space = np.exp(-(space_dist**2) / (2 * config['space_bandwidth']**2))
    weight_time = np.exp(-(time_dist_days**2) / (2 * config['time_bandwidth']**2))
    weight_gaussian = weight_space * weight_time

    # 4. Tính toán trung bình trọng số
    if np.sum(weight_idw) < 1e-9:
        mean_idw = np.nan
    else:
        mean_idw = np.average(values, weights=weight_idw)
        
    if np.sum(weight_gaussian) < 1e-9:
        mean_gaussian = np.nan
    else:
        mean_gaussian = np.average(values, weights=weight_gaussian)
        
    return {
        'xco2_weighted_mean_idw': mean_idw,
        'xco2_weighted_mean_gaussian': mean_gaussian
    }

In [ ]:
config = {
    'time_window': pd.Timedelta(days=30),
    'epsilon': 1e-6, 
    'space_bandwidth': 20.0, 
    'time_bandwidth': 15.0, 
}
weighted_features_df = df.apply(
    lambda row: calculate_spatiotemporal_weighted_features(row, df, config),
    axis=1,
    result_type='expand'
)
df = pd.concat([df, weighted_features_df], axis=1)
print("\n--- DataFrame cuối cùng với đặc trưng có trọng số ---")
print(df.to_string())

## Training 

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def evaluate_model(y_true, y_pred, model_name=""):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"Kết quả cho {model_name}:")
    print(f"  Mean Absolute Error (MAE): {mae:.4f}")
    print(f"  Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"  R-squared (R2): {r2:.4f}\n")

In [ ]:
target = 'xco2'
features = [col for col in df.columns if col not in [target,'time','date', 'xco2_quality_flag','rice_proportion_in_buffer','is_rice_influenced']]
features_to_drop_for_residual = [
    'time_idx',
    'year',  
]
trend_features = ['time_idx']
residual_features = [f for f in features if f not in features_to_drop_for_residual]
print(features)

In [ ]:
split_fraction = 0.8
split_index = int(len(df) * split_fraction)
# train_df = df[df['date'] < '2024-01-01']
# test_df = df[df['date'] >= '2024-01-01']
train_df = df.iloc[:split_index]
test_df = df.iloc[split_index:]

X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features]
y_test = test_df[target]

X_train_trend = X_train[trend_features]
X_test_trend = X_test[trend_features]
X_train_residual = X_train[residual_features]
X_test_residual = X_test[residual_features]

print(f"Kích thước tập Train: {len(X_train)} mẫu")
print(f"Kích thước tập Test: {len(X_test)} mẫu")

### Base models 

In [ ]:
# LightGBM
lgbm = lgb.LGBMRegressor(random_state=42)
lgbm.fit(X_train, y_train)
lgbm_preds = lgbm.predict(X_test)
evaluate_model(y_test, lgbm_preds, "LightGBM cơ bản")

# XGBoost
xgbr = xgb.XGBRegressor(random_state=42)
xgbr.fit(X_train, y_train)
xgbr_preds = xgbr.predict(X_test)
evaluate_model(y_test, xgbr_preds, "XGBoost cơ bản")

# Random Forest
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_test)
evaluate_model(y_test, rf_preds, "Random Forest cơ bản")

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(10, 6))
sns.scatterplot(x=y_test, y=lgbm_preds, ax=ax, alpha=0.6, label='Predicted')
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r', linewidth=2, label='Line (y=x)')
m, c = np.polyfit(y_test, lgbm_preds, 1)
ax.plot(y_test, m * y_test + c, color='blue', linewidth=2, label=f'Fit Line: y={m:.2f}x + {c:.2f}')
r2_value = r2_score(y_test, lgbm_preds)
ax.lines[1].set_label(f'Fit Line (R²={r2_value:.2f})')
ax.set_xlabel("Actual", fontsize=12)
ax.set_ylabel("Predicted", fontsize=12)
ax.set_title("Prediction results on the test set of model LightGBM", fontsize=14)
ax.legend()
ax.grid(True)
plt.show()

### Fine-tune LightGBM

In [ ]:
search_spaces = {
    'num_leaves': Integer(20, 700),
    'n_estimators': Integer(90, 700), 
    'max_depth': Integer(5, 20), 
    'learning_rate': Real(0.01, 0.5, 'log-uniform'),
    'subsample': Real(0.5, 1.0, 'uniform'),
    'colsample_bytree': Real(0.3, 1.0, 'uniform'), 
    'min_child_samples': Integer(5, 30),                   
    'reg_alpha': Real(0.0, 1.0, 'uniform'),           
    'reg_lambda': Real(0.0, 1.0, 'uniform'), 
}
tscv = TimeSeriesSplit(n_splits=5)
lgbm_model = lgb.LGBMRegressor(random_state=42)
bayes_search_lgbm = BayesSearchCV(
    estimator=lgbm_model,
    search_spaces=search_spaces, 
    n_iter=200,  
    cv=tscv,
    scoring='r2',
    verbose=1,
    random_state=42,
    n_jobs=-1
)
bayes_search_lgbm.fit(X_train, y_train)
print(f"\nCác tham số tốt nhất cho LightGBM (Bayesian Optimization): {bayes_search_lgbm.best_params_}")
best_lgbm_params = dict(bayes_search_lgbm.best_params_)

In [ ]:
final_lgbm = lgb.LGBMRegressor(**best_lgbm_params, random_state=42)
final_lgbm.fit(X_train, y_train)
final_preds_lgbm = final_lgbm.predict(X_test)
evaluate_model(y_test, final_preds_lgbm, "LightGBM đã tinh chỉnh")

In [ ]:
importances_lgbm = final_lgbm.feature_importances_
feature_importance_lgbm = pd.DataFrame({
    'Feature': features,
    'Importance': importances_lgbm
})
feature_importance_xgb = feature_importance_lgbm.sort_values(by='Importance', ascending=False)
top_n = 60
top_features_df = feature_importance_xgb.head(top_n)

plt.figure(figsize=(12, 10))
sns.barplot(x='Importance', y='Feature', data=top_features_df, palette='viridis')
plt.title(f'Top {top_n} Feature Importances', fontsize=16)
plt.xlabel('Importance Score', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.tight_layout() 
plt.show()

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(10, 6))
sns.scatterplot(x=y_test, y=final_preds_lgbm, ax=ax, alpha=0.6, label='Predicted')
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r', linewidth=2, label='Line (y=x)')
m, c = np.polyfit(y_test, final_preds_lgbm, 1)
ax.plot(y_test, m * y_test + c, color='blue', linewidth=2, label=f'Fit Line: y={m:.2f}x + {c:.2f}')
r2_value = r2_score(y_test, final_preds_lgbm)
ax.lines[1].set_label(f'Fit Line (R²={r2_value:.2f})')
ax.set_xlabel("Actual", fontsize=12)
ax.set_ylabel("Predicted", fontsize=12)
ax.set_title("Prediction results on the test set of model LightGBM", fontsize=14)
ax.legend()
ax.grid(True)
plt.show()

### Fine-tune XGBoost 

In [ ]:
search_spaces_xgb = {
    'n_estimators': Integer(100, 1000),
    'colsample_bytree': Real(0.3, 1.0, 'uniform'),
    'gamma': Real(0.0, 1.0, 'uniform'),
    'max_depth': Integer(6, 30),
    'subsample': Real(0.5, 1.0, 'uniform'),
    'reg_alpha': Real(0.0, 1.0, 'uniform'),           
    'reg_lambda': Real(0.0, 1.0, 'uniform'),
    'learning_rate': Real(0.01, 0.5, 'log-uniform'),                 
}
tscv = TimeSeriesSplit(n_splits=5)
xgb_model = xgb.XGBRegressor(random_state=42, tree_method='hist')
bayes_search_xgb = BayesSearchCV(
    estimator=xgb_model,
    search_spaces=search_spaces_xgb,
    n_iter=200,
    cv=tscv,
    scoring='r2',
    verbose=1,
    random_state=42,
    n_jobs=1
)
bayes_search_xgb.fit(X_train, y_train)
print(f"\nCác tham số tốt nhất cho XGBoost (Bayesian Optimization): {bayes_search_xgb.best_params_}")
best_xgb_params = dict(bayes_search_xgb.best_params_)

In [ ]:
final_xgb = xgb.XGBRegressor(**best_xgb_params, random_state=42)
final_xgb.fit(X_train, y_train)
final_preds_xgb = final_xgb.predict(X_test)
evaluate_model(y_test, final_preds_xgb, "XGB đã tinh chỉnh")

In [ ]:
importances_xgb = final_xgb.feature_importances_
feature_importance_xgb = pd.DataFrame({
    'Feature': features,
    'Importance': importances_xgb
})
feature_importance_xgb = feature_importance_xgb.sort_values(by='Importance', ascending=False)
top_n = 60
top_features_df = feature_importance_xgb.head(top_n)

plt.figure(figsize=(12, 10))
sns.barplot(x='Importance', y='Feature', data=top_features_df, palette='viridis')
plt.title(f'Top {top_n} Feature Importances (Base on Gain)', fontsize=16)
plt.xlabel('Importance Score', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.tight_layout() 
plt.show()

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(10, 6))
sns.scatterplot(x=y_test, y=final_preds_xgb, ax=ax, alpha=0.6, label='Predicted')
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r', linewidth=2, label='Line (y=x)')
m, c = np.polyfit(y_test, final_preds_xgb, 1)
ax.plot(y_test, m * y_test + c, color='blue', linewidth=2, label=f'Fit Line: y={m:.2f}x + {c:.2f}')
r2_value = r2_score(y_test, final_preds_xgb)
ax.lines[1].set_label(f'Fit Line (R²={r2_value:.2f})')
ax.set_xlabel("Actual", fontsize=12)
ax.set_ylabel("Predicted", fontsize=12)
ax.set_title("Prediction results on the test set of model XGBoost", fontsize=14)
ax.legend()
ax.grid(True)
plt.show()

### Fine-tune Random Forest 

In [ ]:
search_spaces_rf = {
    'n_estimators': Integer(100, 500),
    'max_depth': Integer(6, 30),
    'min_samples_split': Integer(2, 20),
    'min_samples_leaf': Integer(1, 10),    
    'max_features' : Real(0.5, 1.0, 'uniform'),           
}
tscv = TimeSeriesSplit(n_splits=5)
rf_model = RandomForestRegressor(random_state=42)
bayes_search_rf = BayesSearchCV(
    estimator=rf_model,
    search_spaces=search_spaces_rf,
    n_iter=200,
    cv=tscv,
    scoring='r2',
    verbose=1,
    random_state=42,
    n_jobs=1
)
bayes_search_rf.fit(X_train, y_train)
print(f"\nCác tham số tốt nhất cho Random Forest (Bayesian Optimization): {bayes_search_rf.best_params_}")
best_rf_params = dict(bayes_search_rf.best_params_)

In [ ]:
final_rf = RandomForestRegressor(**best_rf_params, random_state=42)
final_rf.fit(X_train, y_train)
final_preds_rf = final_rf.predict(X_test)
evaluate_model(y_test, final_preds_rf, "RF đã tinh chỉnh")

In [ ]:
importances_rf = final_rf.feature_importances_
feature_importance_rf = pd.DataFrame({
    'Feature': features,
    'Importance': importances_rf
})
feature_importance_rf = feature_importance_rf.sort_values(by='Importance', ascending=False)
top_n = 60
top_features_df = feature_importance_xgb.head(top_n)

plt.figure(figsize=(12, 10))
sns.barplot(x='Importance', y='Feature', data=top_features_df, palette='viridis')
plt.title(f'Top {top_n} Feature Importances (Base on Gain)', fontsize=16)
plt.xlabel('Importance Score', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.tight_layout() 
plt.show()

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(10, 6))
sns.scatterplot(x=y_test, y=final_preds_rf, ax=ax, alpha=0.6, label='Predicted')
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r', linewidth=2, label='Line (y=x)')
m, c = np.polyfit(y_test, final_preds_rf, 1)
ax.plot(y_test, m * y_test + c, color='blue', linewidth=2, label=f'Fit Line: y={m:.2f}x + {c:.2f}')
r2_value = r2_score(y_test, final_preds_rf)
ax.lines[1].set_label(f'Fit Line (R²={r2_value:.2f})')
ax.set_xlabel("Actual", fontsize=12)
ax.set_ylabel("Predicted", fontsize=12)
ax.set_title("Prediction results on the test set of model Random Forest", fontsize=14)
ax.legend()
ax.grid(True)
plt.show()

### Fine-tune SVR 

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
search_spaces_svr = {
    'C': Real(1e-1, 1e+3, 'log-uniform'), 
    'gamma': Real(1e-4, 1e+1, 'log-uniform'), 
    'epsilon': Real(0.01, 0.5, 'uniform') 
}
tscv = TimeSeriesSplit(n_splits=5)
bayes_search_svr = BayesSearchCV(
    estimator=SVR(kernel='rbf'),
    search_spaces=search_spaces_svr,
    n_iter=500,  
    cv=tscv,
    scoring='r2',
    verbose=1,
    random_state=42,
    n_jobs=1
)
bayes_search_svr.fit(X_train_scaled, y_train)
print(f"\nCác tham số tốt nhất cho SVR: {bayes_search_svr.best_params_}")
best_svr_params = dict(bayes_search_svr.best_params_)

In [ ]:
final_svr = SVR(kernel='rbf', **best_svr_params)
final_svr.fit(X_train_scaled, y_train)
svr_preds = final_svr.predict(X_test_scaled)
evaluate_model(y_test, svr_preds, "SVR đã tinh chỉnh")

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(10, 6))
sns.scatterplot(x=y_test, y=svr_preds, ax=ax, alpha=0.6, label='Predicted')
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r', linewidth=2, label='Line (y=x)')
m, c = np.polyfit(y_test, svr_preds, 1)
ax.plot(y_test, m * y_test + c, color='blue', linewidth=2, label=f'Fit Line: y={m:.2f}x + {c:.2f}')
r2_value = r2_score(y_test, svr_preds)
ax.lines[1].set_label(f'Fit Line (R²={r2_value:.2f})')
ax.set_xlabel("Actual", fontsize=12)
ax.set_ylabel("Predicted", fontsize=12)
ax.set_title("Prediction results on the test set of model SVR", fontsize=14)
ax.legend()
ax.grid(True)
plt.show()


### Stacking 

In [ ]:
X_train_raw, X_test_raw = df[features].iloc[:split_index], df[features].iloc[split_index:]
y_train, y_test = df[target].iloc[:split_index], df[target].iloc[split_index:]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

X_train = pd.DataFrame(X_train, columns=features, index=X_train_raw.index)
X_test = pd.DataFrame(X_test, columns=features, index=X_test_raw.index)

base_models = {
    'lgbm': lgb.LGBMRegressor(**best_lgbm_params, random_state=42),
    'xgb': xgb.XGBRegressor(**best_xgb_params, random_state=42),
    'rf': RandomForestRegressor(**best_rf_params, random_state=42),
    'svr': SVR(kernel='rbf', **best_svr_params)
}

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
oof_predictions = {name: np.zeros(len(X_train)) for name in base_models.keys()}
oof_indices = []

for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train)):
    print(f"Dang xu ly Fold {fold+1}/5")
    X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    for name, model in base_models.items():
        model.fit(X_train_fold, y_train_fold)
        preds = model.predict(X_val_fold)
        oof_predictions[name][val_idx] = preds
    oof_indices.extend(val_idx)
    
X_train_meta = pd.DataFrame(oof_predictions)
unique_oof_indices = sorted(list(set(oof_indices)))
X_train_meta = X_train_meta.iloc[unique_oof_indices]
y_train_meta = y_train.iloc[unique_oof_indices]

In [ ]:
meta_model = RidgeCV()
meta_model.fit(X_train_meta, y_train_meta)

test_predictions = {}
for name, model in base_models.items():
    model.fit(X_train,y_train)
    test_preds = model.predict(X_test)
    test_predictions[name] = test_preds
    
X_test_meta = pd.DataFrame(test_predictions)

In [ ]:
final_predictions = meta_model.predict(X_test_meta)
evaluate_model(y_test, final_predictions, "Stacking model")

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(10,6))
sns.scatterplot(x=y_test, y=final_predictions, ax=ax, alpha=0.6, label="Predicted")
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r', linewidth=2, label='Line (y=x)')
m, c = np.polyfit(y_test, final_predictions, 1)
ax.plot(y_test, m * y_test + c, color='blue', linewidth=2, label=f'Fit Line: y={m:.2f}x + {c:.2f}')
r2_value = r2_score(y_test, final_predictions)
ax.lines[1].set_label(f'Fit Line (R²={r2_value:.2f})')
ax.set_xlabel("Actual", fontsize=12)
ax.set_ylabel("Predicted", fontsize=12)
ax.set_title("Prediction results on the test set of model Stacking", fontsize=14)
ax.legend()
ax.grid(True)
plt.show()